## Addresses / Wallet

### Generate keys and address

If you already have a payment address and keys and you want to use those, you can skip this step. If not - we need to generate those to submit transactions and to send and receive ada or native assets. Payment verification and signing keys are the first keys we need to create.

In [ ]:
cardano-cli address key-gen \
    --verification-key-file my_address.vkey \
    --signing-key-file my_address.skey

Those two keys can now be used to generate an address.

In [58]:
cardano-cli address build \
    --payment-verification-key-file my_address.vkey \
    --out-file my_address.addr \
    --testnet-magic $CARDANO_NODE_MAGIC

We will save our address hash in a variable called address.

In [60]:
MY_ADDRESS=$(cat my_address.addr)

echo "my address is: $MY_ADDRESS"

my address is: addr_test1vzqf4qj8mj0d7ld9nfcu4s22rhnvx2f0psldjd5gm2z8j6gvqhvde


### Fund the address

Submitting transactions always require you to pay a fee. Sending native assets requires also requires sending at least 1 ada. So make sure the address you are going to use as the input for the minting transaction has sufficient funds.

For the testnet, you can request funds through the [testnet faucet](https://docs.cardano.org/cardano-testnet/tools/faucet).

### Query address funds

Once we've finished requesting funds from the corresponding faucet, we can query the node to ensure that our address now displays the expected amount. For that, we execute the query utxo command using our recently generated address:

In [61]:
cardano-cli query utxo \
    --address $MY_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
fed2616805689edada882eb02c1d70d70f02f12f280c0b3baf6c4c2289204f87     1        9949834279 lovelace + TxOutDatumNone


## Build Transactions

In this tutorial, we'll attempt to create a simple transaction that transfer ADA from one address to another.


### Generate Address for the Destinatary

In part [01](./01-account-management.md) we created our own address and added some funds. Since the goal of this part is to transfer ADA, we'll need to do the same for the destinatary of the transfer.

Run the key-gen command again, but output the keys to different files.

In [62]:
cardano-cli address key-gen \
    --verification-key-file other_address.vkey \
    --signing-key-file other_address.skey

Those two keys can now be used to generate an address.

In [63]:
cardano-cli address build \
    --payment-verification-key-file other_address.vkey \
    --out-file other_address.addr \
    --testnet-magic $CARDANO_NODE_MAGIC

We will save our address hash in a variable called address.

In [64]:
OTHER_ADDRESS=$(cat other_address.addr)

echo "the address of the destinatary is: $OTHER_ADDRESS"

the address of the destinatary is: addr_test1vqw2eumfgg5clc9u8fdhg9g5fl7flalsladpyaz2hku2wugwru2st



### Build the Tx Payload

Txs in Cardano are deterministic. An user building a Tx will need to provide all inputs, outputs, parameters, etc that describe the operation as a whole.

Following the above, the 1st step required to build a Tx is to gather the data of the inputs. For this, lets query the UTxO of our address by running the following command:


In [67]:
cardano-cli query utxo \
    --address $MY_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
fed2616805689edada882eb02c1d70d70f02f12f280c0b3baf6c4c2289204f87     1        9949834279 lovelace + TxOutDatumNone



Since we need each of those values in our transaction, we will store them individually in a corresponding variable.

We also need to define how much we want to transfer. It needs to be less than the amount of available funds, also taking into account the fees for the transfer.

Now we are ready to build the first transaction to calculate our fee and save it in a file called matx.raw. We will reference the variables in our transaction to improve readability because we saved almost all of the needed values in variables. This is what our transaction looks like:

In [70]:
MY_TX_HASH="insert TxHash here"
MY_TX_IX="insert TxIx here"
TRANSFER_AMOUNT="50000000"

cardano-cli transaction build \
 --testnet-magic $CARDANO_NODE_MAGIC \
 --tx-in $MY_TX_HASH#$MY_TX_IX \
 --tx-out $OTHER_ADDRESS+$TRANSFER_AMOUNT \
 --change-address $MY_ADDRESS \
 --out-file transfer.raw

Estimated transaction fee: Lovelace 165721


we can inspect the transaction to ensure that it has what we expect

In [71]:
 cardano-cli transaction view --tx-file transfer.raw

auxiliary scripts: null
certificates: null
collateral inputs: []
era: Babbage
fee: 165721 Lovelace
inputs:
- fed2616805689edada882eb02c1d70d70f02f12f280c0b3baf6c4c2289204f87#1
metadata: null
mint: null
outputs:
- address: addr_test1vqw2eumfgg5clc9u8fdhg9g5fl7flalsladpyaz2hku2wugwru2st
  address era: Shelley
  amount:
    lovelace: 50000000
  datum: null
  network: Testnet
  payment credential key hash: 1cacf36942298fe0bc3a5b7415144ffc9ff7f0ff5a12744abdb8a771
  reference script: null
  stake reference: null
- address: addr_test1vzqf4qj8mj0d7ld9nfcu4s22rhnvx2f0psldjd5gm2z8j6gvqhvde
  address era: Shelley
  amount:
    lovelace: 9899668558
  datum: null
  network: Testnet
  payment credential key hash: 809a8247dc9edf7da59a71cac14a1de6c3292f0c3ed93688da847969
  reference script: null
  stake reference: null
reference inputs: []
required signers (payment key hashes needed for scripts): null
return collateral: null
total collateral: null
update proposal: null
validity range:
  lower bound: n

### Sign the Transaction

Transactions need to be signed to prove the authenticity and ownership of the policy key.

In [72]:
cardano-cli transaction sign  \
    --signing-key-file my_address.skey  \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --tx-body-file transfer.raw  \
    --out-file transfer.signed

jq . transfer.signed

{
  "type": "Witnessed Tx BabbageEra",
  "description": "Ledger Cddl Format",
  "cborHex": "84a30081825820fed2616805689edada882eb02c1d70d70f02f12f280c0b3baf6c4c2289204f87010182a200581d601cacf36942298fe0bc3a5b7415144ffc9ff7f0ff5a12744abdb8a771011a02faf080a200581d60809a8247dc9edf7da59a71cac14a1de6c3292f0c3ed93688da847969011b000000024e10f44e021a00028759a10081825820d308a0c373ce6e915cab83bc913eec5ec74ef3ec055cad5f73d790e4f0b7903758400f34f8e64901752d330aa0aef64750ea39110d4545ed45efceb53e51bf080fc89b6d0a9084b433a8ec14a7a7e6ce76e89a8053e27bc956df0c89d02950f7970ef5f6"
}



### Submit the Transaction

Now we are going to submit the transaction by running the following command:


In [73]:
cardano-cli transaction submit \
    --tx-file transfer.signed \
    --testnet-magic $CARDANO_NODE_MAGIC

Transaction successfully submitted.


Congratulations, we have now successfully transfered some ADA. After a couple of seconds, we can check the output address

In [82]:
cardano-cli query utxo \
    --address $OTHER_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
62a8fc5d8870672fdac1fbbdd209553cdf4334153e67287b98678d6ccf6e00df     0        50000000 lovelace + TxOutDatumNone


## Minting Native Assets

### Create Minting Keys

Policies are the defining factor under which tokens can be minted. Only those in possession of the policy keys can mint or burn tokens minted under this specific policy. We'll make a separate sub-directory in our work directory to keep everything policy-wise separated and more organized. For further reading, please check the official docs or the github page about multi-signature scripts.

First of all, we — again — need some key pairs:

In [83]:
cardano-cli address key-gen \
    --verification-key-file asset_policy.vkey \
    --signing-key-file asset_policy.skey

### Hash the Key

We need to find the hash of the key we just generated, it will be used as data to create the policy script:

In [84]:
POLICY_HASH=$(cardano-cli address key-hash --payment-verification-key-file asset_policy.vkey)

echo "my policy hash is $POLICY_HASH"

my policy hash is 540b538069c3087556293192baf85a4040104404a265933a3fab4c41


### Create a Policy Script

We need to create a file that defines the policy verification key as a witness to sign the minting transaction. There are no further constraints such as token locking or requiring specific signatures to successfully submit a transaction with this minting policy.

Create a new `policy.script` with the following json content, replacing the required values with the data gathered in the previous steps:

```json
{
    "keyHash": "<POLICY_HASH>",
    "type": "sig"
}
```

> **Warning**
> make sure to replace the `<POLICY_HASH>` string with the value computed in the previous step.

In [85]:
echo "{\"type\": \"sig\", \"keyHash\" : \"$POLICY_HASH\" }" > policy.script

jq . policy.script

{
  "type": "sig",
  "keyHash": "540b538069c3087556293192baf85a4040104404a265933a3fab4c41"
}


### Compute the Policy Id

To mint the native assets, we need to generate the policy ID from the script file we created.

In [86]:
POLICY_ID=$(cardano-cli transaction policyid --script-file policy.script)

echo "my policy id is $POLICY_ID"

my policy id is eea157d949cf2d30f16484cb1891a123892667540a2392629ef8f059



The output gets persisted in the `POLICY_ID` variable as we need to reference it later on.



### Define the Token to Mint

We need to defined the name for our token and the amount that we want to mint. Use the following snippet to define the values to use in following steps:

```
TOKEN_NAME="54657374746F6B656E"
TOKEN_AMOUNT="10000000"
```


### Build the Tx

As we saw in part [02](./02-build-transactions.md) of the tutorial, building a transaction requires knowledge of the available UTxO in the address. Use the following command to query our address:

In [87]:
cardano-cli query utxo \
    --address $MY_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
62a8fc5d8870672fdac1fbbdd209553cdf4334153e67287b98678d6ccf6e00df     1        9899668558 lovelace + TxOutDatumNone



> **Note**
> This is the same command we executed when we queried our funds earlier in the tutorial.



Since we need each of those values in our transaction, we will store them individually in a corresponding variable.

In [88]:
TX_HASH="instert TxHash here"
TX_IX="insert TxIx here"
TX_AMOUNT="9899668558"


Our transaction will require to pay fees. For simplicity (and because we're using test ADA), we'll just put a high number to ensure that we cover the required amount:


In [89]:
FEE=200000


To balance the transaction, we need to know which will be the remaining amount of our UTxO once we pay the fees. For that, we calculate the value using the following command:


In [90]:
let REMAINING=$TX_AMOUNT-$FEE


We will reference the variables in our transaction to improve readability because we saved almost all of the needed values in variables. This is what our transaction looks like:


In [92]:
TOKEN_NAME="54657374746F6B656E"
TOKEN_AMOUNT="10000000"

cardano-cli transaction build-raw \
 --fee $FEE \
 --tx-in $TX_HASH#$TX_IX \
 --tx-out $MY_ADDRESS+$REMAINING+"$TOKEN_AMOUNT $POLICY_ID.$TOKEN_NAME" \
 --mint "$TOKEN_AMOUNT $POLICY_ID.$TOKEN_NAME" \
 --minting-script-file policy.script \
 --out-file mint.raw

In [94]:
cardano-cli transaction view --tx-body-file mint.raw

auxiliary scripts: null
certificates: null
collateral inputs: []
era: Babbage
fee: 200000 Lovelace
inputs:
- 62a8fc5d8870672fdac1fbbdd209553cdf4334153e67287b98678d6ccf6e00df#1
metadata: null
mint:
  policy eea157d949cf2d30f16484cb1891a123892667540a2392629ef8f059:
    asset 54657374746f6b656e (Testtoken): 10000000
outputs:
- address: addr_test1vzqf4qj8mj0d7ld9nfcu4s22rhnvx2f0psldjd5gm2z8j6gvqhvde
  address era: Shelley
  amount:
    lovelace: 9899468558
    policy eea157d949cf2d30f16484cb1891a123892667540a2392629ef8f059:
      asset 54657374746f6b656e (Testtoken): 10000000
  datum: null
  network: Testnet
  payment credential key hash: 809a8247dc9edf7da59a71cac14a1de6c3292f0c3ed93688da847969
  reference script: null
  stake reference: null
reference inputs: []
required signers (payment key hashes needed for scripts): null
return collateral: null
total collateral: null
update proposal: null
validity range:
  lower bound: null
  upper bound: null
withdrawals: null



### Sign the Transaction

Transactions need to be signed to prove the authenticity and ownership of the policy key.


In [95]:
cardano-cli transaction sign  \
    --signing-key-file my_address.skey  \
    --signing-key-file asset_policy.skey  \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --tx-body-file mint.raw  \
    --out-file mint.signed


### Submit the Transaction

Now we are going to submit the transaction, therefore minting our native assets:


In [96]:
cardano-cli transaction submit \
    --tx-file mint.signed \
    --testnet-magic $CARDANO_NODE_MAGIC

Transaction successfully submitted.


Congratulations, we have now successfully minted our own token. After a couple of seconds, we can check the output address

In [100]:
cardano-cli query utxo --address $MY_ADDRESS --testnet-magic $CARDANO_NODE_MAGIC

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05faf31b9626187861cf8f567c2ac92a8da525b05a8a9e013f56d41b3a65efed     0        9899468558 lovelace + 10000000 eea157d949cf2d30f16484cb1891a123892667540a2392629ef8f059.54657374746f6b656e + TxOutDatumNone


After a few seconds, you should be able to see that your address holds the newly minted assets